In [ ]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull
import vega_protos.protos.vega as vega_protos

In [ ]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

MM_WALLET = WalletConfig("mm", "mm")

TRADER_1_WALLET = WalletConfig("T1", "pin")

TRADER_2_WALLET = WalletConfig("T2", "pin")

TRADER_A1_WALLET = WalletConfig("A1", "pin")

TRADER_A2_WALLET = WalletConfig("A2", "pin")

TERMINATE_WALLET = WalletConfig("Settle", "pin")

wallets = [MM_WALLET, TRADER_1_WALLET, TRADER_2_WALLET, TRADER_A1_WALLET, TRADER_A2_WALLET, TERMINATE_WALLET]

In [ ]:
vega = VegaServiceNull(run_with_console=False, use_full_vega_wallet=False)
vega.start()

In [ ]:
for wallet in wallets:
    vega.create_wallet(wallet.name, passphrase=wallet.passphrase)
vega.forward('10s')

In [ ]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=10000,
)

vega.forward("20s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
)

tdai_id = vega.find_asset_id(symbol="tDAI")

vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=10e7
)
vega.mint(
    TRADER_1_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_2_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_A1_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_A2_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.wait_for_total_catchup()

In [ ]:
# Create a market
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
        position_decimals=2,
        market_decimals=3,
)

market_id = vega.all_markets()[0].id

vega.submit_simple_liquidity(
    wallet_name=MM_WALLET.name,
    market_id=market_id,
    commitment_amount=600,
    fee=0.05,
    reference_buy="PEGGED_REFERENCE_BEST_BID",
    reference_sell="PEGGED_REFERENCE_BEST_ASK",
    delta_buy=1,
    delta_sell=1,
    is_amendment=False,
)

In [ ]:
# Make market pass opening auction
vega.submit_order(
    trading_wallet=TRADER_A1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=100,
    wait=True,
)
vega.submit_order(
    trading_wallet=TRADER_A2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=100,
    wait=True,
)

# Post limit order
buy_order_id = vega.submit_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=95,
    wait=True,
)

sell_order_id = vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=105,
    wait=True,
)


In [ ]:
print("Margin level of Trader 1 (volume=1):", "\n", vega.margin_levels(wallet_name=TRADER_1_WALLET.name, market_id=market_id))
general, margin, bond = vega.party_account(wallet_name=TRADER_1_WALLET.name, asset_id=tdai_id, market_id=market_id)
print("Margin account: ", margin)

In [ ]:
# Amend order to increase volume
vega.amend_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_id=buy_order_id,
    volume_delta=99,
)

In [ ]:
# Check order info if amend successfully
vega.order_status(order_id=buy_order_id)

In [ ]:
print("Margin level of Trader 1 (volume=100):", "\n", vega.margin_levels(wallet_name=TRADER_1_WALLET.name, market_id=market_id))
general, margin, bond = vega.party_account(wallet_name=TRADER_1_WALLET.name, asset_id=tdai_id, market_id=market_id)
print("Margin account: ", margin)